# ecDNA Detection Pipeline Batch Processing

## Introduction

This Jupyter notebook implements an automated pipeline for detecting and classifying extrachromosomal circular DNA (ecDNA) in Fluorescence in situ Hybridization (FISH) images. The pipeline processes RGB FISH images paired with corresponding DAPI grayscale images to identify and count ecDNA and chromosome-associated objects. Below is a detailed explanation of each component, designed for clarity in supplementary materials

### Datasets

- **RGB Images**: FISH images with probes (e.g., HER2, MYC) stored in `FACS-/proj/brunk_ecdna_cv_project/Poorya/FACS-FISH_redistribution_NCIH2170/version_2_Acc_87/4_batch_process/input_directory/FACS_241119/RGB`.

- **DAPI Images**: Grayscale images highlighting nuclei, stored in `/proj/brunk_ecdna_cv_project/Poorya/FACS-FISH_redistribution_NCIH2170/version_2_Acc_87/4_batch_process/input_directory/FACS_241119/DAPI`.

- **MIA Predicted Images**: Grayscale images predicted by MIA stored in
`/proj/brunk_ecdna_cv_project/Poorya/FACS-FISH_redistribution_NCIH2170/version_2_Acc_87/4_batch_process/input_directory/FACS_241119/MIA`.



### Dependencies

The following libraries are required. Ensure the specified versions are installed for reproducibility:

- `opencv-python==4.1.1`: Image processing.
- `numpy==1.25.0`: Numerical operations.
- `pandas==4.11.0`: Data handling.
- `matplotlib==3.8.4`: Plotting.

Install via:

`pip install opencv-python==4.1.1 numpy==1.25.0 pandas==4.11.0 matplotlib==3.8.4`


In [1]:
# Import Required Libraries
import os
import cv2
import numpy as np
import math
import csv
import json
from multiprocessing import Pool
from functools import partial
import time
import pandas as pd
import matplotlib.pyplot as plt


### Parameter Optimization

The pipeline uses parameters optimized via Bayesian Optimization, stored in best_params.json. If this file is missing, it falls back to default values. These parameters control all aspects of enhancement, detection, and classification, ensuring robustness and accuracy across diverse image sets

In [2]:
# Load Optimized Parameters with Fallback
# Parameters are loaded from 'best_params.json' if available; otherwise, default values are used.
default_params = {
    'kernel_size': 10.83,           # Size of the structuring element for top-hat filtering
    'clip_limit': 3.95,          # Contrast limit for CLAHE
    'tile_grid_size': 21.71,       # Tile size for CLAHE grid
    'strength': 4.666,            # Sharpening strength
    'cutoff': 68.44,              # Sigmoid cutoff value
    'gain': 23.14,                 # Sigmoid gain factor
    'chrom_kernel_size': 168.9,   # Kernel size for chromosome estimation in top-hat
    'dampening_factor': 0.3346,    # Intensity dampening factor for chromosome regions
    'merge_distance': 8.013,       # Distance threshold for merging close objects
    'min_area': 4.133,              # Minimum area for detected objects
    'max_area': 619.6,           # Maximum area for detected objects
    'white_value_threshold': 179.5,  # Brightness threshold for chromosome classification
    'white_saturation_threshold': 63.32  # Saturation threshold for chromosome classification
}

if os.path.exists("best_params.json"):
    with open("best_params.json", "r") as f:
        best_params = json.load(f)
    print("Loaded optimized parameters from best_params.json")
else:
    print("Warning: best_params.json not found, using default parameters")
    best_params = default_params

Loaded optimized parameters from best_params.json


### Input Handling and Preprocessing:

**RGB and DAPI Loading**: RGB images are loaded from a specified folder, and corresponding DAPI images are matched using unique identifiers extracted from filenames.

**Masking:** Pixels in the RGB image where the DAPI image is black (indicating no nuclear material) are set to zero, focusing analysis on regions of interest.

### Note on Commented Code
Print statements and some file-saving operations (e.g., `cv2.imwrite` for final overlays) were commented out to reduce output clutter and disk usage. Uncomment these lines for debugging or to save additional outputs.

## Utility and Helper Functions

In [3]:
def debug_save_image(image, name, step, out_folder, unique_id=""):
    """
    Saves an image to the specified folder with a step number, name, and unique identifier.
    Returns the file path.
    """
    if unique_id:
        filename = f"{unique_id}_{step:02d}_{name}.tif"
    else:
        filename = f"{step:02d}_{name}.tif"
    filepath = os.path.join(out_folder, filename)
    # Save with high quality (no compression for TIFF)
    cv2.imwrite(filepath, image, [int(cv2.IMWRITE_TIFF_COMPRESSION), 1])  # 1 = no compression
    #print(f"Saved: {filepath}")
    return filepath

def extract_unique_id(filename, suffixes):
    """
    Extracts the unique identifier from a filename.
    For example, if filename is "Sample1_Merge.tif" and suffixes is ["_Merge"], 
    then it returns "Sample1".
    We can pass a list of possible suffixes (e.g., ["_Merge", "_DAPI"]).
    """
    base = os.path.splitext(filename)[0]
    for suf in suffixes:
        if base.endswith(suf):
            return base[:-len(suf)]
    # If no suffix is found, return the whole base
    return base

def find_corresponding_dapi(unique_id, dapi_folder):
    """
    Searches the dapi_folder for a file whose unique id (extracted from its name) matches the given unique_id.
    For example, if unique_id is "Sample1", it looks for a file ending with "_DAPI" or "_DAPI_ROI".
    Returns the full path if found; otherwise, returns None.
    """
    # Define possible suffixes for DAPI images
    dapi_suffixes = ["_Merge.tif (RGB)", "_DAPI", "_DAPI.tif", "_Merge.tif (RGB)", "_Merge.tif(RGB)",
                     "_Merge.tif (RGB).tif", "_Merge.tif(RGB).tif","_Merge(RGB)", "_Merge (RGB)",
                     "_Merge(RGB).tif", "_Merge (RGB).tif", "_Merge"]
    for fname in os.listdir(dapi_folder):
        if not fname.lower().endswith(('.tif','.tiff','.png')):
            continue
        uid = extract_unique_id(fname, dapi_suffixes)
        if uid == unique_id:
            return os.path.join(dapi_folder, fname)
    return None

def mask_rgb_with_dapi(rgb_img, dapi_img):
    """
    Given an RGB image and its corresponding DAPI (grayscale) image, set pixels in the RGB image to black 
    wherever the DAPI image is black.
    """
    # Ensure dapi_img is single-channel
    if len(dapi_img.shape) != 2:
        dapi_img = cv2.cvtColor(dapi_img, cv2.COLOR_BGR2GRAY)
    mask = (dapi_img == 0)
    rgb_masked = rgb_img.copy()
    rgb_masked[mask] = 0
    return rgb_masked

##  Image Enhancement Functions

### Image Enhancement:
The RGB image is converted to grayscale to simplify processing.
A four-step enhancement sequence is applied, with parameters optimized via Bayesian Optimization:

- **Top-Hat Filtering:** Enhances small, bright objects (potential ecDNA) using a morphological operation with a structuring element of size `kernel_size`. Chromosome regions are estimated with a larger kernel `(chrom_kernel_size)` and their intensity is reduced by dampening_factor.
- **Sharpening:** Increases edge clarity using a sharpening kernel scaled by `strength`.
- **CLAHE:** Improves local contrast with `clip_limit` and `tile_grid_size`, adapting to varying image conditions.
- **Sigmoid Transformation:** Adjusts intensity using `cutoff` and `gain` to enhance visibility of objects.

In [4]:
def top_hat_enhancement(gray, kernel_size, chrom_kernel_size, dampening_factor):
    """
    Apply top-hat enhancement to emphasize small, bright features while dampening chromosome regions.
    
    Parameters:
    - gray: Grayscale input image
    - kernel_size: Size of the structuring element for top-hat filtering
    - chrom_kernel_size: Size of the structuring element for chromosome estimation
    - dampening_factor: Factor to reduce intensity in chromosome regions
    """
    # Top-hat filtering to enhance small objects
    se = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (int(kernel_size), int(kernel_size)))
    opened = cv2.morphologyEx(gray, cv2.MORPH_OPEN, se)
    top_hat = cv2.subtract(gray, opened)
    top_hat_norm = cv2.normalize(top_hat, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    
    # Estimate chromosome regions and create a mask
    chrom_se = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (int(chrom_kernel_size), int(chrom_kernel_size)))
    chromosome_est = cv2.morphologyEx(gray, cv2.MORPH_CLOSE, chrom_se)
    _, chrom_mask = cv2.threshold(chromosome_est, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # Dampen intensity in chromosome regions
    top_hat_soft = top_hat_norm.astype(np.float32)
    top_hat_soft[chrom_mask == 255] *= dampening_factor
    return np.clip(top_hat_soft, 0, 255).astype(np.uint8)

def apply_sharpening_gray(gray, strength):
    """Apply sharpening to enhance object edges."""
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]], dtype=np.float32) * (strength / 5.0)
    return cv2.filter2D(gray, -1, kernel)

def custom_clahe(gray, clip_limit, tile_grid_size):
    """Apply CLAHE to improve local contrast."""
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=(int(tile_grid_size), int(tile_grid_size)))
    return clahe.apply(gray)

def apply_sigmoid(gray, cutoff, gain):
    """Apply sigmoid transformation for intensity adjustment."""
    norm = gray.astype(np.float32) / 255.0
    c = cutoff / 255.0
    out = 1.0 / (1.0 + np.exp(-gain * (norm - c)))
    return (out * 255).astype(np.uint8)


def process_images(input_image, params, save_debug=False, output_folder=None, unique_id=""):
    """
    Process an input image through the enhancement pipeline: top-hat → sharpening → CLAHE → sigmoid.
    
    Parameters:
    - input_image: Input RGB or grayscale image
    - params: Dictionary of optimized parameters
    - save_debug: Whether to save intermediate images
    - output_folder: Folder for saving debug images
    - unique_id: Unique identifier for the image
    """
    # Convert to grayscale if RGB
    if input_image.ndim == 3:
        gray = cv2.cvtColor(cv2.convertScaleAbs(input_image), cv2.COLOR_BGR2GRAY)
    else:
        gray = cv2.convertScaleAbs(input_image)
    
    simple_gray = gray.copy()
    
    # Enhancement pipeline
    th = top_hat_enhancement(gray, kernel_size=params['kernel_size'], 
                             chrom_kernel_size=params['chrom_kernel_size'], 
                             dampening_factor=params['dampening_factor'])
    sharp = apply_sharpening_gray(th, strength=params['strength'])
    clahe_img = custom_clahe(sharp, clip_limit=params['clip_limit'], 
                             tile_grid_size=params['tile_grid_size'])
    enhanced = apply_sigmoid(clahe_img, cutoff=params['cutoff'], gain=params['gain'])
    debug_save_image(simple_gray, "simple_gray", 1, output_folder, unique_id)
    
    # Save debug images if requested
    if save_debug and output_folder:
        debug_save_image(enhanced, "enhanced_gray", 2, output_folder, unique_id)
    
    return simple_gray, enhanced


##  Object Detection & Overlay Functions

### **Object Detection:**
- **Thresholding:** Applies Otsu’s method to binarize the enhanced image.
- **Morphological Cleaning:** Uses opening and closing operations with a small elliptical kernel to remove noise and connect fragmented objects.
- **Connected Components:** Identifies objects, filtering by area (`min_area` to `max_area`).
- **Merging:** Combines objects closer than `merge_distance` based on centroid distance.

### Classification and Annotation:

- **Classification:** Objects are classified as `“chromosome”` or `“ecDNA”` using the RGB image’s HSV values. Regions with high brightness `(white_value_threshold)` and low saturation `(white_saturation_threshold)` are labeled as chromosomes; others are ecDNA.
- **Annotation:** Bounding boxes are drawn on both the enhanced grayscale and RGB images, with counts recorded.

In [5]:
def merge_close_objects(objects, merge_distance):
    """Merge objects that are closer than the specified distance."""
    merged_objects = []
    taken = [False] * len(objects)
    for i in range(len(objects)):
        if taken[i]:
            continue
        current = objects[i].copy()
        for j in range(i + 1, len(objects)):
            if taken[j]:
                continue
            if math.dist(current["centroid"], objects[j]["centroid"]) < merge_distance:
                x1, y1, w1, h1 = current["bbox"]
                x2, y2, w2, h2 = objects[j]["bbox"]
                current["bbox"] = (min(x1, x2), min(y1, y2), max(x1 + w1, x2 + w2) - min(x1, x2), 
                                   max(y1 + h1, y2 + h2) - min(y1, y2))
                cx1, cy1 = current["centroid"]
                cx2, cy2 = objects[j]["centroid"]
                current["centroid"] = ((cx1 + cx2) / 2.0, (cy1 + cy2) / 2.0)
                current["area"] += objects[j]["area"]
                taken[j] = True
        merged_objects.append(current)
        taken[i] = True
    return merged_objects

def classify_as_white_or_ecDNA(roi, white_value_threshold, white_saturation_threshold):
    """Classify an object as 'chromosome' or 'ecDNA' based on HSV values."""
    hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    _, S_mean, V_mean, _ = cv2.mean(hsv_roi)
    return "chromosome" if (V_mean > white_value_threshold and S_mean < white_saturation_threshold) else "ecDNA"



def annotate_objects(merged_objects, rgb_image_path, simple_gray, output_folder, unique_id, params):
    """
    Annotate detected objects on both the RGB and simple grayscale images, and count classifications.

    Parameters:
    - merged_objects (list): List of detected objects with bbox, centroid, and area.
    - rgb_image_path (str): Path to the RGB image.
    - simple_gray (numpy.ndarray): Simple grayscale image to annotate.
    - output_folder (str): Folder to save annotated images.
    - unique_id (str): Unique identifier for the image.
    - params (dict): Dictionary of optimized parameters.

    Returns:
    - dict: Counts of chromosomes and ecDNA.
    """
    # Load and annotate the RGB image
    rgb_img = cv2.imread(rgb_image_path, cv2.IMREAD_COLOR)
    if rgb_img is None:
        print(f"Could not load RGB image: {rgb_image_path}")
        return {}

    # Ensure simple_gray is a valid grayscale image
    if simple_gray is None or len(simple_gray.shape) != 2:
        print(f"Invalid simple grayscale image for {unique_id}. Skipping grayscale annotation.")
        annotated_gray = None
    else:
        # Convert grayscale to BGR for colored annotations
        annotated_gray = cv2.cvtColor(simple_gray, cv2.COLOR_GRAY2BGR)

    # Initialize counts
    counts = {"chromosome": 0, "ecDNA": 0}
    marker_size = 6  # Size of the top-left corner marker

    # Use the original RGB image without enhancement
    annotated_rgb = rgb_img.copy()

    # Create an overlay layer for semi-transparent annotations
    overlay_rgb = np.zeros_like(annotated_rgb, dtype=np.uint8)
    if annotated_gray is not None:
        overlay_gray = np.zeros_like(annotated_gray, dtype=np.uint8)

    # Annotate the RGB image
    for obj in merged_objects:
        x, y, w, h = obj["bbox"]
        roi = rgb_img[y:y+h, x:x+w]
        label = classify_as_white_or_ecDNA(roi, params['white_value_threshold'], params['white_saturation_threshold'])
        counts[label] += 1

        # Set color: green for ecDNA, white for chromosomes
        color = (0, 255, 0) if label == "ecDNA" else (255, 255, 255)

        # Draw bounding box on overlay (thinner line)
        cv2.rectangle(overlay_rgb, (x, y), (x+w, y+h), color, 1)

        # Draw centroid (smaller)
        cx, cy = obj["centroid"]
        cv2.circle(overlay_rgb, (int(cx), int(cy)), 1, (0, 0, 255), -1)

        # Add text label for chromosomes (smaller font)
        if label == "chromosome":
            cv2.putText(overlay_rgb, "chromosome", (x, y - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.4, color, 1)

        # Draw marker in top-left corner (smaller and on overlay)
        for mx in range(x, x + marker_size):
            for my in range(y, y + marker_size):
                if mx < overlay_rgb.shape[1] and my < overlay_rgb.shape[0]:
                    overlay_rgb[my, mx] = color

    # Apply alpha blending for RGB annotations
    alpha = 0.35  # Transparency factor (0.0 = fully transparent, 1.0 = fully opaque)
    annotated_rgb = cv2.addWeighted(annotated_rgb, 1.0, overlay_rgb, alpha, 0.0)

    # Save the annotated RGB image with high quality
    debug_save_image(annotated_rgb, "annotated_rgb", 5, output_folder, unique_id)

    # Annotate the grayscale image if available
    if annotated_gray is not None:
        for obj in merged_objects:
            x, y, w, h = obj["bbox"]
            label = classify_as_white_or_ecDNA(rgb_img[y:y+h, x:x+w], 
                                               params['white_value_threshold'], 
                                               params['white_saturation_threshold'])

            # Set color: green for ecDNA, white for chromosomes
            color = (0, 255, 0) if label == "ecDNA" else (255, 255, 255)

            # Draw bounding box on overlay (thinner line)
            cv2.rectangle(overlay_gray, (x, y), (x+w, y+h), color, 1)

            # Draw centroid (smaller)
            cx, cy = obj["centroid"]
            cv2.circle(overlay_gray, (int(cx), int(cy)), 1, (0, 0, 255), -1)

            # Add text label for chromosomes (smaller font)
            if label == "chromosome":
                cv2.putText(overlay_gray, "chromosome", (x, y - 10), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.4, color, 1)

            # Draw marker in top-left corner (smaller and on overlay)
            for mx in range(x, x + marker_size):
                for my in range(y, y + marker_size):
                    if mx < overlay_gray.shape[1] and my < overlay_gray.shape[0]:
                        overlay_gray[my, mx] = color

        # Apply alpha blending for grayscale annotations
        annotated_gray = cv2.addWeighted(annotated_gray, 1.0, overlay_gray, alpha, 0.0)

        # Save the annotated grayscale image with high quality
        debug_save_image(annotated_gray, "annotated_gray", 6, output_folder, unique_id)

    return counts

def object_detection_and_overlay(enhanced_img, rgb_img_path, simple_gray, output_folder, unique_id, params):
    """
    Detect objects in the enhanced image and overlay them on both grayscale and RGB images.
    
    Parameters:
    - enhanced_img: Enhanced grayscale image
    - rgb_img_path: Path to the RGB image
    - simple_gray: Simple grayscale image to annotate
    - output_folder: Folder for saving output images
    - unique_id: Unique identifier for the image
    - params: Dictionary of optimized parameters
    """
    # Threshold and clean the image
    _, thresh = cv2.threshold(enhanced_img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2, 2))
    cleaned = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
    cleaned = cv2.morphologyEx(cleaned, cv2.MORPH_CLOSE, kernel)
    
    # Detect connected components
    num_labels, _, stats, centroids = cv2.connectedComponentsWithStats(cleaned, connectivity=8)
    objects = [{"bbox": (stats[i, 0], stats[i, 1], stats[i, 2], stats[i, 3]), 
                "area": stats[i, 4], 
                "centroid": centroids[i]}
               for i in range(1, num_labels) if params['min_area'] <= stats[i, 4] <= params['max_area']]
    
    # Merge close objects
    merged_objects = merge_close_objects(objects, merge_distance=params['merge_distance'])
    
    # Overlay on enhanced grayscale
    enhanced_color = cv2.cvtColor(enhanced_img, cv2.COLOR_GRAY2BGR)
    for obj in merged_objects:
        x, y, w, h = obj["bbox"]
        cv2.rectangle(enhanced_color, (x, y), (x+w, y+h), (0, 255, 0), 1)
    debug_save_image(enhanced_color, "overlay_on_gray", 4, output_folder, unique_id)
    
    # Annotate on both RGB and simple grayscale images
    counts = annotate_objects(merged_objects, rgb_img_path, simple_gray, output_folder, unique_id, params)
    
    return len(merged_objects), merged_objects, counts



def convert_obj(obj):
    """Convert object data to a JSON-serializable format."""
    return {
        "bbox": tuple(int(x) for x in obj["bbox"]),
        "centroid": tuple(float(x) for x in obj["centroid"]),
        "area": float(obj["area"])
    }

## Main Processing Pipeline (First Part)

In [6]:
def process_single_image(args, params, rgb_folder, dapi_folder, output_folder):
    """
    Process a single RGB image and its corresponding DAPI image.

    Parameters:
    - args (tuple): Contains (fname, dapi_path)
    - params (dict): Optimized parameters
    - rgb_folder (str): Path to RGB images
    - dapi_folder (str): Path to DAPI images
    - output_folder (str): Path to save outputs

    Returns:
    - dict: Metrics for the image (filename, unique_id, Total, chromosome, ecDNA)
    - tuple: (unique_id, ref_objects) for storing reference data
    """
    fname, dapi_path = args
    try:
        rgb_path = os.path.join(rgb_folder, fname)
        rgb_img = cv2.imread(rgb_path, cv2.IMREAD_COLOR)
        if rgb_img is None or rgb_img.ndim != 3:
            print(f"Skipping {fname}: Not a valid RGB image.")
            return None, None

        # Extract unique id
        rgb_suffixes = ["_Merge.tif (RGB)", "_DAPI", "_DAPI.tif", "_Merge.tif (RGB)", "_Merge.tif(RGB)",
                        "_Merge.tif (RGB).tif", "_Merge.tif(RGB).tif","_Merge(RGB)", "_Merge (RGB)",
                        "_Merge(RGB).tif", "_Merge (RGB).tif", "_Merge", "_Merge ", "_Merge.tif"]
        unique_id = extract_unique_id(fname, rgb_suffixes)

        # Load and mask with DAPI
        if dapi_path is not None:
            dapi_img = cv2.imread(dapi_path, cv2.IMREAD_GRAYSCALE)
            if dapi_img is not None:
                new_rgb = mask_rgb_with_dapi(rgb_img, dapi_img)
            else:
                print(f"Could not load DAPI image for {unique_id}. Using original RGB.")
                new_rgb = rgb_img  # Avoid copy if not necessary
        else:
            print(f"No corresponding DAPI found for {unique_id}.")
            new_rgb = rgb_img

        # Save new RGB
        new_rgb_path = os.path.join(output_folder, f"{unique_id}_1_newRGB.tif")
        cv2.imwrite(new_rgb_path, new_rgb)

        # Process the image
        simple_gray, enhanced = process_images(new_rgb, params, save_debug=False, 
                                               output_folder=output_folder, unique_id=unique_id)

        # Detect and annotate objects, passing simple_gray
        total_count, merged_objects, counts = object_detection_and_overlay(
            enhanced, new_rgb_path, simple_gray, output_folder, unique_id, params
        )

        # Store reference data
        ref_objects = [convert_obj(o) for o in merged_objects]

        # Return metrics and reference data
        metrics = {
            'filename': fname,
            'unique_id': unique_id,
            'Total': total_count,
            'chromosome': counts.get("chromosome", 0),
            'ecDNA': counts.get("ecDNA", 0)
        }
        return metrics, (unique_id, ref_objects)

    except Exception as e:
        print(f"Error processing {fname}: {e}")
        return None, None
    
    
    

def main_pipeline(rgb_folder, dapi_folder, output_folder, metrics_csv_path, params):
    """
    Main pipeline to process RGB and DAPI images in parallel, detect objects, and save metrics.

    Parameters:
    - rgb_folder: Folder containing RGB images
    - dapi_folder: Folder containing DAPI images
    - output_folder: Folder to save output images and metrics
    - metrics_csv_path: Path to save the metrics CSV file
    - params: Dictionary of optimized parameters
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Preload all RGB files and their corresponding DAPI paths
    image_pairs = []
    rgb_files = [fname for fname in os.listdir(rgb_folder) if fname.lower().endswith(('.tif', '.tiff', '.png'))]
    for fname in rgb_files:
        unique_id = extract_unique_id(fname, ["_Merge.tif (RGB)", "_DAPI", "_DAPI.tif", "_Merge.tif (RGB)",
                                              "_Merge.tif(RGB)", "_Merge.tif (RGB).tif", "_Merge.tif(RGB).tif",
                                             "_Merge(RGB)", "_Merge (RGB)", "_Merge(RGB).tif", "_Merge (RGB).tif", "_Merge"])
        dapi_path = find_corresponding_dapi(unique_id, dapi_folder)
        image_pairs.append((fname, dapi_path))

    # Set up parallel processing
    num_workers = min(os.cpu_count(), 8)  # Limit to 8 workers to manage memory usage
    print(f"Using {num_workers} workers for parallel processing.")

    # Process images in parallel
    process_func = partial(process_single_image, params=params, rgb_folder=rgb_folder, 
                           dapi_folder=dapi_folder, output_folder=output_folder)
    
    metrics_list = []
    ref_data = {}
    with Pool(processes=num_workers) as pool:
        results = pool.map(process_func, image_pairs)
    
    # Collect results
    for metrics, ref_entry in results:
        if metrics is not None:
            metrics_list.append(metrics)
        if ref_entry is not None:
            unique_id, ref_objects = ref_entry
            ref_data[unique_id] = ref_objects

    # Write metrics to CSV all at once
    csv_path = os.path.join(output_folder, "metrics_summary.csv")
    with open(csv_path, 'w', newline='') as csvfile:
        fieldnames = ['filename', 'unique_id', 'Total', 'chromosome', 'ecDNA']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for metrics in metrics_list:
            writer.writerow(metrics)

    # Save reference objects to JSON
    json_path = os.path.join(output_folder, "ref_objects.json")
    with open(json_path, "w") as jf:
        json.dump(ref_data, jf)

    print(f"Processing complete. Metrics saved to {csv_path}")

### Output:

- **Metrics:** Total object count and counts of chromosomes and ecDNA are saved in a CSV file (metrics_summary.csv).
- **Reference Data:** Object details (bounding boxes, centroids, areas) are saved in a JSON file (ref_objects.json).
- **Debug Images:** Intermediate images (e.g., masked RGB, enhanced grayscale, annotated RGB) are saved with step numbers for validation.

In [7]:
# Run the Pipeline
rgb_folder = r"/proj/brunk_ecdna_cv_project/Poorya/FACS-FISH_redistribution_NCIH2170/version_2_Acc_87/4_batch_process/input_directory/FACS_241119/RGB"  # Replace with your RGB image folder path
dapi_folder = r"/proj/brunk_ecdna_cv_project/Poorya/FACS-FISH_redistribution_NCIH2170/version_2_Acc_87/4_batch_process/input_directory/FACS_241119/DAPI"  # Replace with your DAPI image folder path
output_folder = r"/proj/brunk_ecdna_cv_project/Poorya/FACS-FISH_redistribution_NCIH2170/version_2_Acc_87/4_batch_process/Final_Output/FACS_241119_Output"  # Replace with your output folder path
metrics_csv_path = os.path.join(output_folder, "metrics_summary.csv")

start_time = time.time()
main_pipeline(rgb_folder, dapi_folder, output_folder, metrics_csv_path, best_params)
print(f"First part runtime: {time.time() - start_time:.2f} seconds")

Using 8 workers for parallel processing.
No corresponding DAPI found for H2170_TOTAL_HER2_G2_2502_125.
No corresponding DAPI found for H2170_HIGH_HER2_G0_2411_27.
No corresponding DAPI found for H2170_TOTAL_HER2_G2_2502_52.
Processing complete. Metrics saved to /proj/brunk_ecdna_cv_project/Poorya/FACS-FISH_redistribution_NCIH2170/version_2_Acc_87/4_batch_process/Final_Output/FACS_241119_Output/metrics_summary.csv
First part runtime: 2989.35 seconds


In [8]:
import os
import cv2
import json
import csv
import math
import numpy as np

##############################
#  Helper & Segmentation Functions
##############################

def debug_save_image(image, name, step, out_folder, unique_id=""):
    """Save image to disk with a step number and unique identifier."""
    if unique_id:
        filename = f"{unique_id}_{step:02d}_{name}.png"
    else:
        filename = f"{step:02d}_{name}.png"
    filepath = os.path.join(out_folder, filename)
    cv2.imwrite(filepath, image)
    #print(f"Saved: {filepath}")
    return filepath

def compute_iou(bboxA, bboxB):
    """Compute Intersection over Union (IoU) for two bounding boxes."""
    xA = max(bboxA[0], bboxB[0])
    yA = max(bboxA[1], bboxB[1])
    xB = min(bboxA[0] + bboxA[2], bboxB[0] + bboxB[2])
    yB = min(bboxA[1] + bboxA[3], bboxB[1] + bboxB[3])
    if xB < xA or yB < yA:
        return 0.0
    interArea = (xB - xA) * (yB - yA)
    boxAArea = bboxA[2] * bboxA[3]
    boxBArea = bboxB[2] * bboxB[3]
    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

def match_with_partial(ref_objects, mia_objects, iou_full=0.01, iou_partial=0.001, distance_threshold=30.0):
    """
    Match objects between reference and MIA predictions.
    Returns:
      full_matches: list of (ref_index, mia_index) where IoU >= iou_full.
      partial_matches: list of (ref_index, mia_index) where IoU between iou_partial and iou_full or centroids are very close.
      unmatched_ref: set of indices in ref_objects not matched.
      unmatched_mia: set of indices in mia_objects not matched.
    """
    full_matches = []
    partial_matches = []
    unmatched_ref = set(range(len(ref_objects)))
    unmatched_mia = set(range(len(mia_objects)))
    
    for i, ref_obj in enumerate(ref_objects):
        ref_bbox = ref_obj["bbox"]
        for j, mia_obj in enumerate(mia_objects):
            if j not in unmatched_mia:
                continue
            mia_bbox = mia_obj["bbox"]
            iou = compute_iou(ref_bbox, mia_bbox)
            if iou >= iou_full:
                full_matches.append((i, j))
                unmatched_ref.discard(i)
                unmatched_mia.discard(j)
                break
            elif iou >= iou_partial:
                partial_matches.append((i, j))
                unmatched_ref.discard(i)
                unmatched_mia.discard(j)
                break
            else:
                dist = math.dist(ref_obj["centroid"], mia_obj["centroid"])
                if dist < distance_threshold:
                    partial_matches.append((i, j))
                    unmatched_ref.discard(i)
                    unmatched_mia.discard(j)
                    break
    return full_matches, partial_matches, unmatched_ref, unmatched_mia

def compute_metrics(full_matches, partial_matches, unmatched_ref, unmatched_mia):
    """Compute TP, partial TP, FP, and FN from matching results."""
    TP = len(full_matches)
    TP_partial = len(partial_matches)
    FN = len(unmatched_ref)
    FP = len(unmatched_mia)
    return TP, TP_partial, FP, FN

def segment_mia_mask(mia_mask_path, min_area=1, max_area=500, unique_id=""):
    """
    Segments the MIA predicted mask:
      1. Loads the binary MIA mask.
      2. Applies morphological closing to fill holes.
      3. Extracts connected components.
    Returns a list of detected predicted objects.
    """
    mia_mask = cv2.imread(mia_mask_path, cv2.IMREAD_GRAYSCALE)
    if mia_mask is None:
        raise IOError(f"Could not load MIA mask: {mia_mask_path}")
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    mia_filled = cv2.morphologyEx(mia_mask, cv2.MORPH_CLOSE, kernel)
    
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(mia_filled, connectivity=8)
    ecDNA_mia_objects = []
    for label_id in range(1, num_labels):  # skip background
        x, y, w, h, area = stats[label_id]
        cx, cy = centroids[label_id]
        if area < min_area or area > max_area:
            continue
        ecDNA_mia_objects.append({
            "label_id": label_id,
            "bbox": (x, y, w, h),
            "area": area,
            "centroid": (cx, cy)
        })
    return ecDNA_mia_objects, labels

def overlay_two_images(image1, image2, alpha=0.5):
    """Blend two images using alpha blending."""
    return cv2.addWeighted(image1, alpha, image2, 1 - alpha, 0)

def visualize_matches(ref_image_path, ref_objects, mia_objects, 
                      full_matches, partial_matches, unmatched_ref, unmatched_mia, 
                      out_folder, unique_id=""):
    """
    Creates an overlay visualization on the reference image:
      - Green: Full matches (TP)
      - Cyan: Partial matches
      - Red: False negatives (reference objects missed by MIA)
      - Blue: False positives (extra objects from MIA predictions)
    Saves the final overlay image with matching details.
    """
    # Load the reference image (we assume it's grayscale; if not, adjust accordingly)
    base_img = cv2.imread(ref_image_path, cv2.IMREAD_GRAYSCALE)
    if base_img is None:
        print("Error: Could not load reference image for visualization.")
        return
    # Convert to color so we can draw colored markers
    vis_img = cv2.cvtColor(base_img, cv2.COLOR_GRAY2BGR)
    
    # Draw full matches (TP) in green
    for (i, j) in full_matches:
        cx, cy = ref_objects[i]["centroid"]
        cv2.circle(vis_img, (int(cx), int(cy)), 5, (0, 255, 0), -1)
    
    # Draw partial matches in cyan
    for (i, j) in partial_matches:
        cx, cy = ref_objects[i]["centroid"]
        cv2.circle(vis_img, (int(cx), int(cy)), 5, (255, 255, 0), -1)
    
    # Draw false negatives (reference objects missed) in red
    for i in unmatched_ref:
        cx, cy = ref_objects[i]["centroid"]
        cv2.circle(vis_img, (int(cx), int(cy)), 5, (0, 0, 255), -1)
    
    # Draw false positives (extra MIA predictions) in blue
    for j in unmatched_mia:
        cx, cy = mia_objects[j]["centroid"]
        cv2.circle(vis_img, (int(cx), int(cy)), 5, (255, 0, 0), -1)
    
    # Save the visualization image
    vis_path = os.path.join(out_folder, f"{unique_id}_final_matches.tif")
    cv2.imwrite(vis_path, vis_img)
    #print(f"Saved detailed matching visualization: {vis_path}")

##############################
#  Second-Part Pipeline
##############################

def update_metrics_with_mia(existing_csv, ref_objects_json, output_folder, mia_folder, updated_csv_path,
                            ref_overlay_gray_suffix="_04_overlay_on_gray.tif",
                            ref_annotated_rgb_suffix="_05_annotated_rgb.tif"):
    """
    For each image (row) from the existing CSV (from part 1):
      - Use the unique_id to locate the saved reference overlays.
      - Load the corresponding reference objects from ref_objects_json.
      - Locate and segment the MIA predicted image (without enhancing it).
      - Match MIA objects to the stored reference objects.
      - Compute metrics: TP, TP_partial, FN, FP, and MIA_total.
      - Create final overlays by blending the MIA mask with the reference overlays.
      - Update the CSV row with the new metrics.
      - Write an updated copy of the CSV.
    """
    # Load existing CSV rows
    rows = []
    with open(existing_csv, 'r', newline='') as f:
        reader = csv.DictReader(f)
        for row in reader:
            rows.append(row)
    
    # Load stored reference objects (from part 1)
    with open(ref_objects_json, 'r') as jf:
        ref_data = json.load(jf)  # expected format: { unique_id: [ {bbox, centroid, area}, ... ], ... }
    
    for row in rows:
        unique_id = row.get("unique_id", "").strip()
        if not unique_id:
            # If missing, derive from filename
            fname = row.get("filename", "")
            unique_id = os.path.splitext(fname)[0].strip()
            row["unique_id"] = unique_id
        
        print(f"Processing unique_id: '{unique_id}'")
        
        # Get stored reference objects
        if unique_id not in ref_data:
            print(f"No reference objects for {unique_id}. Skipping update for this entry.")
            continue
        ref_objects = ref_data[unique_id]
        
        # Build expected file paths for the reference overlay images
        ref_gray_path = os.path.join(output_folder, f"{unique_id}{ref_overlay_gray_suffix}")
        ref_rgb_path = os.path.join(output_folder, f"{unique_id}{ref_annotated_rgb_suffix}")
        #print(f"Looking for ref_gray: '{ref_gray_path}'")
        #print(f"Looking for ref_rgb: '{ref_rgb_path}'")
        if not os.path.exists(ref_gray_path) or not os.path.exists(ref_rgb_path):
            print(f"Missing reference overlays for {unique_id}. Skipping update for this entry.")
            continue
        
        # Find corresponding MIA predicted image from mia_folder
        mia_path = None
        for fname in os.listdir(mia_folder):
            if fname.lower().endswith(('.tif','.tiff','.png')) and unique_id in fname:
                mia_path = os.path.join(mia_folder, fname)
                break
        if not mia_path:
            print(f"No MIA predicted image for {unique_id}. Skipping update for this entry.")
            continue
        
        # Segment the MIA predicted image
        try:
            mia_objects, _ = segment_mia_mask(mia_path, min_area=1, max_area=500, unique_id=unique_id)
        except Exception as e:
            print(f"Error segmenting MIA image for {unique_id}: {e}")
            continue
        mia_total = len(mia_objects)
        
        # Compute matching between stored reference objects and MIA objects
        full_matches, partial_matches, unmatched_ref, unmatched_mia = match_with_partial(
            ref_objects, mia_objects, iou_full=0.01, iou_partial=0.001, distance_threshold=30.0)
        TP, TP_partial, FP, FN = compute_metrics(full_matches, partial_matches, unmatched_ref, unmatched_mia)

        # Visualize matches
        visualize_matches(ref_gray_path, ref_objects, mia_objects,
                          full_matches, partial_matches, unmatched_ref, unmatched_mia,
                          output_folder, unique_id)
        
        # Load the reference overlay images and the MIA image for overlay visualization
        ref_gray_img = cv2.imread(ref_gray_path, cv2.IMREAD_GRAYSCALE)
        ref_rgb_img = cv2.imread(ref_rgb_path, cv2.IMREAD_COLOR)
        mia_img = cv2.imread(mia_path, cv2.IMREAD_GRAYSCALE)
        if ref_gray_img is None or ref_rgb_img is None or mia_img is None:
            print(f"Error loading images for {unique_id}. Skipping update for this entry.")
            continue
        
        # Create final overlays by blending the MIA predicted mask with each reference overlay
        ref_gray_color = cv2.cvtColor(ref_gray_img, cv2.COLOR_GRAY2BGR)
        mia_color = cv2.cvtColor(mia_img, cv2.COLOR_GRAY2BGR)
        final_overlay_gray = overlay_two_images(ref_gray_color, mia_color, alpha=0.7)
        final_overlay_gray_path = os.path.join(output_folder, f"{unique_id}_07_final_overlay_gray.tif")
        #cv2.imwrite(final_overlay_gray_path, final_overlay_gray)
        
        final_overlay_rgb = overlay_two_images(ref_rgb_img, mia_color, alpha=0.7)
        final_overlay_rgb_path = os.path.join(output_folder, f"{unique_id}_08_final_overlay_rgb.tif")
        #cv2.imwrite(final_overlay_rgb_path, final_overlay_rgb)
        
        # Update the CSV row with new metrics
        row["MIA_total"] = mia_total
        row["TP"] = TP
        row["TP_partial"] = TP_partial
        row["FN"] = FN
        row["FP"] = FP
        row["ref_total"] = len(ref_objects)
        
        print(f"Updated {unique_id}: ref_total={len(ref_objects)}, MIA_total={mia_total}, TP={TP}, TP_partial={TP_partial}, FN={FN}, FP={FP}")
    
    # Write updated CSV (copy of part-1 CSV updated)
    with open(updated_csv_path, 'w', newline='') as f:
        fieldnames = rows[0].keys()
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        for r in rows:
            writer.writerow(r)
    
    print(f"MIA validation update complete. Updated CSV saved to {updated_csv_path}")

In [9]:
# Define the folder paths (update these to match your environment):
output_folder = r"/proj/brunk_ecdna_cv_project/Poorya/FACS-FISH_redistribution_NCIH2170/version_2_Acc_87/4_batch_process/Final_Output/FACS_241119_Output"     # Folder containing your overlay images
mia_folder = r"/proj/brunk_ecdna_cv_project/Poorya/FACS-FISH_redistribution_NCIH2170/version_2_Acc_87/4_batch_process/input_directory/FACS_241119/MIA"       # Folder with MIA predicted images
existing_csv = os.path.join(output_folder, "metrics_summary.csv")  # CSV produced in part 1
ref_objects_json = os.path.join(output_folder, "ref_objects.json")  # The JSON file you saved from part 1
updated_csv_path = os.path.join(output_folder, "updated_ecDNA_counts.csv")

# Now call the update pipeline:
start_time = time.time()
update_metrics_with_mia(existing_csv, ref_objects_json, output_folder, mia_folder, updated_csv_path,
                        ref_overlay_gray_suffix="_04_overlay_on_gray.tif",
                        ref_annotated_rgb_suffix="_05_annotated_rgb.tif")
print(f"Second part runtime: {time.time() - start_time:.2f} seconds")

Processing unique_id: 'H2170_HIGH_HER2_G1_2501_1'
Updated H2170_HIGH_HER2_G1_2501_1: ref_total=180, MIA_total=99, TP=3, TP_partial=34, FN=143, FP=62
Processing unique_id: 'H2170_LOW_HER2_G2_2502_11'
Updated H2170_LOW_HER2_G2_2502_11: ref_total=103, MIA_total=101, TP=1, TP_partial=14, FN=88, FP=86
Processing unique_id: 'H2170_LOW_HER2_G0_2411_80'
Updated H2170_LOW_HER2_G0_2411_80: ref_total=103, MIA_total=44, TP=13, TP_partial=17, FN=73, FP=14
Processing unique_id: 'H2170_TOTAL_HER2_G1_2501_55'
Updated H2170_TOTAL_HER2_G1_2501_55: ref_total=250, MIA_total=90, TP=13, TP_partial=77, FN=160, FP=0
Processing unique_id: 'H2170_LOW_HER2_G2_2502_8'
Updated H2170_LOW_HER2_G2_2502_8: ref_total=223, MIA_total=67, TP=0, TP_partial=39, FN=184, FP=28
Processing unique_id: 'H2170_TOTAL_HER2_G0_2411_92'
Updated H2170_TOTAL_HER2_G0_2411_92: ref_total=370, MIA_total=188, TP=112, TP_partial=62, FN=196, FP=14
Processing unique_id: 'H2170_LOW_HER2_G1_2501_111'
Updated H2170_LOW_HER2_G1_2501_111: ref_total=

In [10]:
# Load the updated CSV
df = pd.read_csv(updated_csv_path)

# Compute summary statistics
summary = {
    "Total Images": len(df),
    "Average ref_total": df["ref_total"].mean(),
    "Average MIA_total": df["MIA_total"].mean(),
    "Average TP": df["TP"].mean(),
    "Average TP_partial": df["TP_partial"].mean(),
    "Average FN": df["FN"].mean(),
    "Average FP": df["FP"].mean(),
}

# Display summary
print("Summary Statistics:")
for key, value in summary.items():
    print(f"{key}: {value:.2f}")

Summary Statistics:
Total Images: 1206.00
Average ref_total: 254.56
Average MIA_total: 123.65
Average TP: 53.46
Average TP_partial: 54.53
Average FN: 146.56
Average FP: 15.66


In [11]:
import os
import shutil

# Set the source directory
source_dir = output_folder

# Define the main groups and subcategories.
groups = {
    "_G0_": "G0",
    "_G1_": "G1",
    "_G2_": "G2"
}
subcategories = {
    "_HIGH_": "High_HER2",
    "_LOW_": "Low_HER2",
    "_TOTAL_": "Total_HER2"
}

# Create the folder structure if it doesn't already exist
for group_folder in groups.values():
    for sub_folder in subcategories.values():
        os.makedirs(os.path.join(source_dir, group_folder, sub_folder), exist_ok=True)

# Iterate over each file in the source directory
for filename in os.listdir(source_dir):
    file_path = os.path.join(source_dir, filename)
    
    # Skip directories to only process files
    if not os.path.isfile(file_path):
        continue

    # Determine the group folder based on filename content
    group_folder = None
    for key, folder in groups.items():
        if key in filename:
            group_folder = folder
            break
    
    # If no group was found, skip this file
    if group_folder is None:
        continue

    # Determine the subcategory folder based on filename content
    sub_folder = None
    for key, folder in subcategories.items():
        if key in filename:
            sub_folder = folder
            break
    
    # If no subcategory was found, skip this file
    if sub_folder is None:
        continue

    # Build the destination path
    dest_path = os.path.join(source_dir, group_folder, sub_folder, filename)
    
    # Move the file to the new folder
    shutil.move(file_path, dest_path)
    #print(f"Moved {filename} to {dest_path}")


# Create a Summary folder for CSV and JSON files
summary_folder = os.path.join(source_dir, "Summary")
os.makedirs(summary_folder, exist_ok=True)

# Move CSV and JSON files
for file in ["metrics_summary.csv", "updated_ecDNA_counts.csv", "ref_objects.json"]:
    src = os.path.join(source_dir, file)
    dst = os.path.join(summary_folder, file)
    if os.path.exists(src):
        shutil.move(src, dst)
        print(f"Moved {file} to {dst}")

Moved metrics_summary.csv to /proj/brunk_ecdna_cv_project/Poorya/FACS-FISH_redistribution_NCIH2170/version_2_Acc_87/4_batch_process/Final_Output/FACS_241119_Output/Summary/metrics_summary.csv
Moved updated_ecDNA_counts.csv to /proj/brunk_ecdna_cv_project/Poorya/FACS-FISH_redistribution_NCIH2170/version_2_Acc_87/4_batch_process/Final_Output/FACS_241119_Output/Summary/updated_ecDNA_counts.csv
Moved ref_objects.json to /proj/brunk_ecdna_cv_project/Poorya/FACS-FISH_redistribution_NCIH2170/version_2_Acc_87/4_batch_process/Final_Output/FACS_241119_Output/Summary/ref_objects.json


## Acknowledgment

**Use of AI-Based Code Generation and Editing Tools**  
This project was enhanced by the use of AI tools—ChatGPT, Grok, and Copilot—which played significant roles in improving the code, explanations, and overall presentation of the work.

### Specific Contributions

- **ChatGPT**:
  - Added comments to various functions to improve code readability and understanding.
  - Assisted in optimizing functions and replacing less effective ones, leading to better performance and results.
  - Provided clear, simplified explanations of complex concepts to make the content more accessible.
  - Edited text for grammatical accuracy, clarity, and an appropriate tone.

- **Grok**:
  - Contributed to adding comments to functions for better documentation.
  - Helped optimize functions and suggested replacements that improved the project’s outcomes.
  - Assisted in refining explanations to ensure they were concise and easy to understand.


All three tools—ChatGPT, Grok, and Copilot—were used to simplify explanations, making them clearer and more accessible. They also assisted in editing the grammar and tone of the text to improve readability and professionalism.

- **Iterative Refinement Process:**
For most parts of this project, I provided the same prompts to ChatGPT, Grok, and Copilot to compare their responses. I then asked each tool to evaluate the outputs of the others and provide feedback. This feedback was shared back with the tools, allowing them to refine their suggestions iteratively. This process ensured that the final code, explanations, and results were optimized for accuracy, clarity, and effectiveness.

### Citations

- OpenAI. (2023). ChatGPT (Version o3-mini-high for code and logics & Version 4.5 for text editing) Large language model. https://openai.com/chatgpt
- Grok (2023). xAI.(Version 3.0). Used for text generation and code refinement in this project. https://xai.com/grok
- Microsoft Copilot (an AI assistant developed by Microsoft),



